In [30]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 16 09:13:21 2024
@author: 
"""

from mpi4py import MPI
import timeit
import meshio
COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

from manapy.ddm import readmesh
from manapy.ddm import Domain

from manapy.tools.pyccel_tools import initialisation_gaussian_2d, update_new_value, time_step
                          
from manapy.fvm.pyccel_fvm import (explicitscheme_convective_2d,
                                   explicitscheme_dissipative)

#from manapy.ast import Variable, LinearSystem

import numpy as np
import os

start = timeit.default_timer()

# mesh directory

dim = 2
readmesh('rect.msh', dim=dim, periodic=[0,0,0])

#Create the informations about cells, faces and nodes
domain = Domain(dim=dim)

faces = domain.faces
cells = domain.cells
halos = domain.halos
nodes = domain.nodes

nbnodes = domain.nbnodes
nbfaces = domain.nbfaces
nbcells = domain.nbcells


Starting ....
Number of Cells :  1720
Number of Nodes :  911


In [31]:
domain.nbcells

1720

In [32]:
#Cells Centre
Cells_centre= [cell[:-1] for cell in cells._center]

In [33]:
import numpy.linalg as LA

def circumcenter(C):
    ax = C[0][0]
    ay = C[0][1]
    bx = C[1][0]
    by = C[1][1]
    cx = C[2][0]
    cy = C[2][1]
    d = 2 * (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by))
    ux = ((ax * ax + ay * ay) * (by - cy) + (bx * bx + by * by) * (cy - ay) + (cx * cx + cy * cy) * (ay - by)) / d
    uy = ((ax * ax + ay * ay) * (cx - bx) + (bx * bx + by * by) * (ax - cx) + (cx * cx + cy * cy) * (bx - ax)) / d
    return (ux, uy)

def dist(x, y):
    x=np.asarray(x)
    y=np.asarray(y)
    return LA.norm(x - y)

In [34]:
def norm2(exact, Sol, volume, nbelements, order):      
    Error = np.zeros(nbelements)
    Ex = np.zeros(nbelements)
    for i in range(nbelements):
        Error[i] = np.fabs(Sol[i]- exact[i]) * volume[i]
        Ex[i] = np.fabs(exact[i]) * volume[i]

    ErrorL2 = np.linalg.norm(Error,ord=order)/np.linalg.norm(Ex,ord=order)
    return ErrorL2

In [35]:
def cell_ver_coor():
    Cells_Cord=[]
    for s in cells._nodeid :
        Cell_Cord =[]
        for i in s :
            Cell_Cord.append(nodes._vertex[i][:-2])
        Cells_Cord.append(Cell_Cord)   
    return Cells_Cord

Cells_Cord = cell_ver_coor()

In [36]:
#Orthocentre
Orthocentre = []
for i in range(nbcells):
    Orthocentre.append(circumcenter(Cells_Cord[i]))

In [37]:
def save_paraview_one_variable(w, cells, nodes, dim, name):

    if dim == 2:
        elements = {"triangle": cells}
    elif dim == 3:
        elements = {"tetra": cells}

    points = []
    for i in nodes:
        points.append([i[0], i[1], i[2]])

    cells  = np.array(cells)
    points = np.array(points)

    data = {"w" : w}

    if len(w) == len(cells):
        data = {"w": data}

    if len(w) == len(cells):
        meshio.write_points_cells("visu"+name+".vtu",
                                    points, elements, cell_data=data, file_format="vtu")
    else:
        meshio.write_points_cells("visu"+str(w)+".vtu",
                                    points, elements, point_data=data, file_format="vtu")


In [38]:
dist(Orthocentre[0],Orthocentre[1])

1.4770561880469253

In [39]:
import numpy as np
def Sol_exact(x, y, alpha = 4):
    return np.exp(-1*(x**2+y**2)/0.25**2)


u_exact = [ Sol_exact(cell[0], cell[1]) for cell in domain.cells.center]
u_exact = np.array(u_exact)

save_n=0
save_paraview_one_variable(u_exact, cells._nodeid, nodes._vertex, 2, "res_VF4"+str(save_n))

In [40]:
cells.faceid[0]

array([ 921, 1636,  917])

In [41]:
target_cells=2
cell1 = faces.cellid[cells.faceid[target_cells]][0][1]
cell2 = faces.cellid[cells.faceid[target_cells]][1][1]
cell3 = faces.cellid[cells.faceid[target_cells]][2][1]

cell1,cell1,cell1

(482, 482, 482)

In [42]:
target_cells=0

cells.faceid[target_cells]

array([ 921, 1636,  917])

In [43]:
face=cells.faceid[target_cells][0]
k = faces.cellid[face][0]
l = faces.cellid[face][1]
k,l
face

921

In [44]:
mesure = faces.mesure[face]
mesure

0.19300810014671296

In [45]:
dist1 = dist(Orthocentre[k], Orthocentre[l])
dist1

0.08601681555623067

In [46]:
##---------------------Computing of dt-------------##
somme = []
for i in range(nbcells):
    k = cells.volume[i]
    list_faces = list(cells.faceid[i])
    elem = 0
    for j in range(3):
        mesure1 = faces.mesure[list_faces[j]]
        dist1 = dist(Orthocentre[list(faces.cellid[list_faces[j]])[0]],Orthocentre[list(faces.cellid[list_faces[j]])[1]])
        elem += mesure1/dist1
    somme += [1/k*elem]

lamda = 0.8
D = 1

dt = lamda/(D * max (somme))

print(dt)


temps=0
T=0.15
u=u_exact
i=0
unew = np.zeros(nbcells)
while temps<=T:
    temps+=dt
    save_n+=1
    for j in range(nbcells):
        list_faces = cells.faceid[j]
        elem = 0
        for k in range(3):
            mesure1 = faces.mesure[list_faces[k]]
            dist1 = dist(Orthocentre[faces.cellid[list_faces[k]][0]],Orthocentre[faces.cellid[list_faces[k]][1]])
            if j == faces.cellid[list_faces[k]][0]:
                u_diff = u[faces.cellid[list_faces[k]][1]] - u[faces.cellid[list_faces[k]][0]]
            else :
                u_diff = u[faces.cellid[list_faces[k]][0]] - u[faces.cellid[list_faces[k]][1]]
            elem += mesure1*u_diff/dist1
        unew[j] = u[j] - dt/(cells.volume[j]) * (-D)*elem

    u=np.copy(unew)
    i+=1
    
    save_paraview_one_variable(u, cells._nodeid, nodes._vertex, 2, "res_VF4"+str(save_n))

0.00016159281287490268


KeyboardInterrupt: 

In [ ]:
#####sudo apt-get install paraview